# Data exploration

In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import torch

In [2]:
mt_path = os.path.join(os.path.abspath('../'), 'participants_data_v2021', 'mini_track')
ft_path = os.path.join(os.path.abspath('../'), 'participants_data_v2021', 'full_track')

## MiniTrack

In [3]:
for folder in sorted(os.listdir(mt_path)):
    print(folder)

sub01
sub02
sub03
sub04
sub05
sub06
sub07
sub08
sub09
sub10


In [4]:
sub_1_EBA = pickle.load(open(os.path.join(mt_path, 'sub01', 'EBA.pkl'), 'rb'))

In [5]:
sub_1_EBA.keys()

dict_keys(['train'])

In [6]:
sub_1_EBA['train'][0]

array([[ 0.25350905, -0.57726972, -0.72763889, ..., -0.53123644,
        -0.34700944, -0.87076372],
       [ 0.05788916,  0.63083047,  0.23448168, ..., -0.64568988,
         0.6862889 ,  1.375431  ],
       [ 0.4457479 ,  0.53338382,  0.8011759 , ..., -0.07184202,
        -0.22447055,  1.25935497]])

In [7]:
sub_1_EBA['train'].shape

(1000, 3, 351)

In [8]:
sensenet = torch.load('../sense/resources/backbone/strided_inflated_efficientnet.ckpt')

In [9]:
# sensenet.keys()

### Results

In [10]:
import sys
sys.path.append('../')

In [11]:
from perform_encoding import get_fmri
from utils.ols import vectorized_correlation

/home/spanchal/miniconda3/envs/Algonauts2021/lib/python3.7/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [12]:
ROIs = ['LOC', 'FFA', 'STS', 'EBA', 'PPA', 'V1', 'V2', 'V3', 'V4']
num_subs = 10

results_folder = os.path.join(os.path.abspath('../'), 'results')

result_keys = ['alexnet_devkit_5', 'sense_full', 'sense_32']
result_dirs = [
    'alexnet_devkit/layer_5/mini_track',
    'sense_efficientnet/mini_track/full',
    'sense_efficientnet/mini_track/pca_32'
]
result_dirs = [os.path.join(results_folder, dir_) for dir_ in result_dirs]
fmri_dir = os.path.join(os.path.abspath('../'), 'participants_data_v2021', 'mini_track')

data = {}

for result_key, result_dir in zip(result_keys, result_dirs):
    data[result_key] = {}
    
    for roi in ROIs:
        data[result_key][roi] = []
        data[result_key][f"{roi}_score"] = []
        
        for sub in range(1, 11):
            sub = f"sub{sub:02}"
            sub_fmri_dir = os.path.join(fmri_dir, sub)
            
            all_fmri_data = get_fmri(sub_fmri_dir, roi)
            
            fmri_test = all_fmri_data[900:, :]
            pred_fmri = np.load(os.path.join(result_dir, sub, f"{roi}_val.npy"))
            
            data[result_key][roi].append(pred_fmri)
            data[result_key][f"{roi}_score"].append(round(vectorized_correlation(fmri_test, pred_fmri).mean(), 3))

In [13]:
np.mean(data['alexnet_devkit_5']['LOC_score'])

0.1607

In [14]:
data_reformated = {'ROIs': ROIs}
data_reformated.update({
    results_key: [
        round(np.mean(data[results_key][f'{roi}_score']), 3) for roi in ROIs
    ] for results_key in data
})

In [15]:
df = pd.DataFrame(data_reformated, columns=data_reformated.keys())

#### Validation results

In [16]:
df

,ROIs,alexnet_devkit_5,sense_full,sense_32
0,LOC,0.161,-0.021,0.175
1,FFA,0.147,-0.033,0.148
2,STS,0.087,-0.011,0.100
3,EBA,0.176,-0.039,0.218
4,PPA,0.097,0.015,0.102
5,V1,0.184,-0.011,0.082
6,V2,0.170,-0.012,0.089
7,V3,0.150,-0.010,0.103
8,V4,0.122,-0.004,0.102
